
# STRUCTURE

In [8]:
import ipyrad.analysis as ipa
import toyplot
import ipyparallel as ipp

In [2]:
## look for running ipcluster instance, and create load-balancer
ipyclient = ipp.Client()
print("{} engines found".format(len(ipyclient)))

24 engines found


In [9]:
# the path to your HDF5 formatted snps file
data = "./T_ae_8rm_v9.snps.hdf5"

In [10]:
imap = {
#"ref": ["reference"],
#"Inam": ["T_ae_T14205_In", "T_ae_A7968_In", "T_ae_A8087_In", "T_ae_28095_In", "T_ae_75520_In"],
#"Puru": ["T_ae_T12313_pu", "T_ae_T13219_pu", "T_ae_A3559_pu", "T_ae_A439_pu", "T_ae_82491_pu", "T_ae_82606_pu", "T_ae_82608_pu", "T_ae_A21562_pu", "T_ae_A2720_pu", "T_ae_A2833_pu"], 
"JiGu": ["T_ae_T3376_jigu","T_ae_T3237_jigu", "T_ae_A317_jigu", "T_ae_A324_jigu"],#, "T_ae_T3260_jigu" removed because clusters with Inam/Purus
"Mach": ["T_ae_T13278_ma", "T_ae_A458_ma", "T_ae_T2166_ma", "T_ae_T2207_ma", "T_ae_T4355_ma", "T_ae_A509_ma", "T_ae_J249_ma", "T_ae_J252_ma", "T_ae_J261_ma", "T_ae_J298_ma"],
"Roar": ["T_ae_J319_roar", "T_ae_J419_roar", "T_ae_J678_roar", "T_ae_J683_roar"],
"ArSu": ["T_ae_80508_arsu", "T_ae_80716_arsu", "T_ae_81278_arsu",   "T_ae_A522_ma",  "T_ae_86147_arsu", "T_ae_86229_arsu",  "T_ae_J598_arsu", "T_ae_J616_arsu","T_ae_86569_arsu","T_ae_81338_arsu"],
"SuTa": ["T_ae_T14601_suta", "T_ae_T16625_suta", "T_ae_T16704_suta", "T_ae_85274_suta", "T_ae_A15069_suta", "T_ae_T742_suta", "T_ae_T10229_suta", "T_ae_T19436_suta", "T_ae_A16041_suta","T_ae_T7941_suta","T_ae_T24591_suta"],
"Para": ["T_ae_A15279_pa", "T_ae_T10679_pa", "T_ae_T13575_pa", "T_ae_T17307_pa", "T_ae_T19420_pa", "T_ae_T8268_pa"]
}

# minimum % of samples that must be present in each SNP from each group
minmap = {i: 0.5 for i in imap}

In [5]:
# init analysis object with input data and (optional) parameter options
struct = ipa.structure(
    name="T_ae_str_east",
    data=data,
    imap=imap,
    minmap=minmap,
    mincov=0.95,
)

Samples: 45
Sites before filtering: 1641207
Filtered (indels): 0
Filtered (bi-allel): 36142
Filtered (mincov): 1606581
Filtered (minmap): 1638777
Filtered (combined): 1638871
Sites after filtering: 2336
Sites containing missing values: 0 (0.00%)
Missing values in SNP matrix: 0 (0.00%)


In [6]:
struct.mainparams.burnin = 50000
struct.mainparams.numreps = 200000
struct.write_structure_files(abs)

('/array1/lmusher/rio_roosevelt_outfiles/T_ae_8rm_v9_outfiles/analysis-structure/tmp-T_ae_str_east-<built-in function abs>-1.mainparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/T_ae_8rm_v9_outfiles/analysis-structure/tmp-T_ae_str_east-<built-in function abs>-1.extraparams.txt',
 '/array1/lmusher/rio_roosevelt_outfiles/T_ae_8rm_v9_outfiles/analysis-structure/tmp-T_ae_str_east-<built-in function abs>-1.strfile.txt')

In [7]:
struct.run(nreps=10, kpop=[2, 3, 4, 5, 6, 7, 8], ipyclient=ipyclient, force=True)

Parallel connection | amnh-gen-001.internal.amnh.org: 24 cores
[####################] 100% 0:20:29 | running 70 structure jobs 


In [11]:
struct = ipa.structure(
    data=data, 
    name="T_ae_str_east", 
    workdir="analysis-structure",
    imap=imap,
    load_only=True,
)

70 previous results loaded for run [T_ae_str_east]


In [12]:
etable = struct.get_evanno_table([2, 3, 4, 5, 6, 7, 8])
etable

,Nreps,deltaK,estLnProbMean,estLnProbStdev,lnPK,lnPPK
2,10,0.000,-2182.56,177.967,0.00,0.00
3,10,0.963,-2105.16,57.871,77.40,55.73
4,10,0.056,-2083.49,122.726,21.67,6.88
5,10,1.169,-2068.70,168.445,14.79,196.93
6,10,0.719,-2250.84,541.991,-182.14,389.86
7,10,2.285,-2043.12,100.544,207.72,229.76
8,10,0.000,-2065.16,150.221,-22.04,0.00


In [13]:
# get canvas object and set size
canvas = toyplot.Canvas(width=400, height=300)

# plot the mean log probability of the models in red
axes = canvas.cartesian(ylabel="estLnProbMean")
axes.plot(etable.estLnProbMean * -1, color="darkred", marker="o")
axes.y.spine.style = {"stroke": "darkred"}

# plot delta K with its own scale bar of left side and in blue
axes = axes.share("x", ylabel="deltaK", ymax=etable.deltaK.max() + etable.deltaK.max() * .25)
axes.plot(etable.deltaK, color="steelblue", marker="o");
axes.y.spine.style = {"stroke": "steelblue"}

# set x labels
axes.x.ticks.locator = toyplot.locator.Explicit(range(len(etable.index)), etable.index)
axes.x.label.text = "K (N ancestral populations)"

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t8f0e47c015cc44b5b2cbacef5cf506c0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 2 3 4 5 6 7 8 K (N ancestral populations) 2050 2100 2150 2200 2250 estLnProbMean 0 1 2 3 deltaK

In [14]:
k = 2
table = struct.get_clumpp_table(k)

[K2] 10/10 results permuted across replicates (max_var=0).


In [15]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [16]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t43da535de55a45e0bfb2acb27c1a2350" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> T_ae_A15279_pa T_ae_T10679_pa T_ae_T13575_pa T_ae_T17307_pa T_ae_T19420_pa T_ae_T8268_pa T_ae_J319_roar T_ae_J419_roar T_ae_J678_roar T_ae_J683_roar T_ae_80508_arsu T_ae_80716_arsu T_ae_81278_arsu T_ae_A522_ma T_ae_86147_arsu T_ae_86229_arsu T_ae_J598_arsu T_ae_J616_arsu T_ae_86569_arsu T_ae_81338_arsu T_ae_T14601_suta T_ae_T16625_suta T_ae_T16704_suta T_ae_85274_suta T_ae_A15069_suta T_ae_T742_suta T_ae_T10229_suta T_ae_T19436_suta T_ae_A16041_suta T_ae_T7941_suta T_ae_T24591_suta T_ae_T3376_jigu T_ae_T3237_jigu T_ae_A317_jigu T_ae_A324_jigu T_ae_T13278_ma T_ae_A458_ma T_ae_T2166_ma T_ae_T2207_ma T_ae_T4355_ma T_ae_A509_ma T_ae_J249_ma T_ae_J252_ma T_ae_J261_ma T_ae_J298_ma 0.0 0.5 1.0

In [17]:
k = 3
table = struct.get_clumpp_table(k)

[K3] 10/10 results permuted across replicates (max_var=0).


In [18]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [19]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="ta3a8c85019df40caacb67a1582af8993" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> T_ae_A15279_pa T_ae_T10679_pa T_ae_T13575_pa T_ae_T17307_pa T_ae_T19420_pa T_ae_T8268_pa T_ae_J319_roar T_ae_J419_roar T_ae_J678_roar T_ae_J683_roar T_ae_80508_arsu T_ae_80716_arsu T_ae_81278_arsu T_ae_A522_ma T_ae_86147_arsu T_ae_86229_arsu T_ae_J598_arsu T_ae_J616_arsu T_ae_86569_arsu T_ae_81338_arsu T_ae_T14601_suta T_ae_T16625_suta T_ae_T16704_suta T_ae_85274_suta T_ae_A15069_suta T_ae_T742_suta T_ae_T10229_suta T_ae_T19436_suta T_ae_A16041_suta T_ae_T7941_suta T_ae_T24591_suta T_ae_T3376_jigu T_ae_T3237_jigu T_ae_A317_jigu T_ae_A324_jigu T_ae_T13278_ma T_ae_A458_ma T_ae_T2166_ma T_ae_T2207_ma T_ae_T4355_ma T_ae_A509_ma T_ae_J249_ma T_ae_J252_ma T_ae_J261_ma T_ae_J298_ma 0.0 0.5 1.0

In [20]:
k = 4
table = struct.get_clumpp_table(k)

[K4] 10/10 results permuted across replicates (max_var=0).


In [21]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
onames = ["T_ae_T3376_jigu","T_ae_T3237_jigu", "T_ae_A317_jigu", "T_ae_A324_jigu","T_ae_T13278_ma", "T_ae_A458_ma", "T_ae_T2166_ma", "T_ae_T2207_ma", "T_ae_T4355_ma", "T_ae_A509_ma", "T_ae_J249_ma", "T_ae_J252_ma", "T_ae_J261_ma", "T_ae_J298_ma","T_ae_J319_roar", "T_ae_J419_roar", "T_ae_J678_roar", "T_ae_J683_roar","T_ae_80508_arsu", "T_ae_80716_arsu", "T_ae_81278_arsu",   "T_ae_A522_ma",  "T_ae_86147_arsu", "T_ae_86229_arsu",  "T_ae_J598_arsu", "T_ae_J616_arsu","T_ae_86569_arsu","T_ae_81338_arsu","T_ae_T14601_suta", "T_ae_T16625_suta", "T_ae_T16704_suta", "T_ae_85274_suta", "T_ae_A15069_suta", "T_ae_T742_suta", "T_ae_T10229_suta", "T_ae_T19436_suta", "T_ae_A16041_suta","T_ae_T7941_suta","T_ae_T24591_suta","T_ae_A15279_pa", "T_ae_T10679_pa", "T_ae_T13575_pa", "T_ae_T17307_pa", "T_ae_T19420_pa", "T_ae_T8268_pa"]
table = table.loc[onames]

In [22]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tc06ef1f7b288447a9f42d2f81890267f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> T_ae_T3376_jigu T_ae_T3237_jigu T_ae_A317_jigu T_ae_A324_jigu T_ae_T13278_ma T_ae_A458_ma T_ae_T2166_ma T_ae_T2207_ma T_ae_T4355_ma T_ae_A509_ma T_ae_J249_ma T_ae_J252_ma T_ae_J261_ma T_ae_J298_ma T_ae_J319_roar T_ae_J419_roar T_ae_J678_roar T_ae_J683_roar T_ae_80508_arsu T_ae_80716_arsu T_ae_81278_arsu T_ae_A522_ma T_ae_86147_arsu T_ae_86229_arsu T_ae_J598_arsu T_ae_J616_arsu T_ae_86569_arsu T_ae_81338_arsu T_ae_T14601_suta T_ae_T16625_suta T_ae_T16704_suta T_ae_85274_suta T_ae_A15069_suta T_ae_T742_suta T_ae_T10229_suta T_ae_T19436_suta T_ae_A16041_suta T_ae_T7941_suta T_ae_T24591_suta T_ae_A15279_pa T_ae_T10679_pa T_ae_T13575_pa T_ae_T17307_pa T_ae_T19420_pa T_ae_T8268_pa 0.0 0.5 1.0

In [23]:
k = 5
table = struct.get_clumpp_table(k)

[K5] 10/10 results permuted across replicates (max_var=0).


In [24]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
import itertools
onames = list(itertools.chain(*imap.values()))
table = table.loc[onames]

In [25]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="tecf03acdf4454286a5a3e8da0d0c7e7f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> T_ae_A15279_pa T_ae_T10679_pa T_ae_T13575_pa T_ae_T17307_pa T_ae_T19420_pa T_ae_T8268_pa T_ae_J319_roar T_ae_J419_roar T_ae_J678_roar T_ae_J683_roar T_ae_80508_arsu T_ae_80716_arsu T_ae_81278_arsu T_ae_A522_ma T_ae_86147_arsu T_ae_86229_arsu T_ae_J598_arsu T_ae_J616_arsu T_ae_86569_arsu T_ae_81338_arsu T_ae_T14601_suta T_ae_T16625_suta T_ae_T16704_suta T_ae_85274_suta T_ae_A15069_suta T_ae_T742_suta T_ae_T10229_suta T_ae_T19436_suta T_ae_A16041_suta T_ae_T7941_suta T_ae_T24591_suta T_ae_T3376_jigu T_ae_T3237_jigu T_ae_A317_jigu T_ae_A324_jigu T_ae_T13278_ma T_ae_A458_ma T_ae_T2166_ma T_ae_T2207_ma T_ae_T4355_ma T_ae_A509_ma T_ae_J249_ma T_ae_J252_ma T_ae_J261_ma T_ae_J298_ma 0.0 0.5 1.0

In [26]:
k = 7
table = struct.get_clumpp_table(k)

[K7] 10/10 results permuted across replicates (max_var=0).


In [27]:
# sort list by columns
table.sort_values(by=list(range(k)), inplace=True)

# or, sort by a list of names (here taken from imap)
onames = ["T_ae_T3376_jigu","T_ae_T3237_jigu", "T_ae_A317_jigu", "T_ae_A324_jigu","T_ae_T13278_ma", "T_ae_A458_ma", "T_ae_T2166_ma", "T_ae_T2207_ma", "T_ae_T4355_ma", "T_ae_A509_ma", "T_ae_J249_ma", "T_ae_J252_ma", "T_ae_J261_ma", "T_ae_J298_ma","T_ae_J319_roar", "T_ae_J419_roar", "T_ae_J678_roar", "T_ae_J683_roar","T_ae_80508_arsu", "T_ae_80716_arsu", "T_ae_81278_arsu",   "T_ae_A522_ma",  "T_ae_86147_arsu", "T_ae_86229_arsu",  "T_ae_J598_arsu", "T_ae_J616_arsu","T_ae_86569_arsu","T_ae_81338_arsu","T_ae_T14601_suta", "T_ae_T16625_suta", "T_ae_T16704_suta", "T_ae_85274_suta", "T_ae_A15069_suta", "T_ae_T742_suta", "T_ae_T10229_suta", "T_ae_T19436_suta", "T_ae_A16041_suta","T_ae_T7941_suta","T_ae_T24591_suta","T_ae_A15279_pa", "T_ae_T10679_pa", "T_ae_T13575_pa", "T_ae_T17307_pa", "T_ae_T19420_pa", "T_ae_T8268_pa"]
table = table.loc[onames]

In [28]:
# build barplot
canvas = toyplot.Canvas(width=500, height=250)
axes = canvas.cartesian(bounds=("10%", "90%", "10%", "45%"))
axes.bars(table)

# add labels to x-axis
ticklabels = [i for i in table.index.tolist()]
axes.x.ticks.locator = toyplot.locator.Explicit(labels=ticklabels)
axes.x.ticks.labels.angle = -60
axes.x.ticks.show = True
axes.x.ticks.labels.offset = 10
axes.x.ticks.labels.style = {"font-size": "12px"}

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t9c43d6c2287645efb727cf8bda4c4257" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 250.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> T_ae_T3376_jigu T_ae_T3237_jigu T_ae_A317_jigu T_ae_A324_jigu T_ae_T13278_ma T_ae_A458_ma T_ae_T2166_ma T_ae_T2207_ma T_ae_T4355_ma T_ae_A509_ma T_ae_J249_ma T_ae_J252_ma T_ae_J261_ma T_ae_J298_ma T_ae_J319_roar T_ae_J419_roar T_ae_J678_roar T_ae_J683_roar T_ae_80508_arsu T_ae_80716_arsu T_ae_81278_arsu T_ae_A522_ma T_ae_86147_arsu T_ae_86229_arsu T_ae_J598_arsu T_ae_J616_arsu T_ae_86569_arsu T_ae_81338_arsu T_ae_T14601_suta T_ae_T16625_suta T_ae_T16704_suta T_ae_85274_suta T_ae_A15069_suta T_ae_T742_suta T_ae_T10229_suta T_ae_T19436_suta T_ae_A16041_suta T_ae_T7941_suta T_ae_T24591_suta T_ae_A15279_pa T_ae_T10679_pa T_ae_T13575_pa T_ae_T17307_pa T_ae_T19420_pa T_ae_T8268_pa 0.0 0.5 1.0